In [ ]:
!pip install torch transformers diffusers faiss-cpu pillow accelerate torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 32.3 MB/s eta 0:00:00


## Data Preparation and Embedding

In [ ]:
import os
import json
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import faiss
import numpy as np
import pickle

# Define the data
DATA = [
    {"image": "lrrh_title1.jpg",       "text": "Little Red Riding Hood"},
    {"image": "lrrh_mother2.jpg",      "text": "Long long ago, in a small house near the forest, there lived a little girl with her mother. Her name was Little Red Riding Hood. One day her mother said, “Grandma is sick. She’s in bed. Go to her house. In the basket, there’s a bottle of fresh milk, some bread and some butter for her.”"},
    {"image": "lrrh_forest3.jpg",   "text": "Little Red Riding Hood walked through the forest carrying the basket. Suddenly a big bad wolf appeared. “Where are you going, little girl?” asked the wolf. “To see my grandmother,” said innocent Little Red Riding Hood; “she’s sick. I have some milk, bread and butter for her.” “Pick some flowers,” said the Wolf. And he ran to grandmother’s house"},
    {"image": "lrrh_granwolf4.jpg", "text": "The Wolf got to grandmother’s house. He went tap-tap-tap on the door. “Who is this?” asked Grandma. “It’s me, Little Red Riding Hood,” said the Wolf; “I have some milk, bread and butter for you.” “Open the door and come in, my dear,” said Grandma. The Wolf opened the door and, in an instant, he ate her up! Then, he got into grandma’s bed to wait for Little Red Riding Hood. Little Red Riding Hood got to grandmother’s house. She went tap-tap-tap on the door. “Who is this?” asked the Wolf. “It’s me, Little Red Riding Hood. I have some milk, bread and butter for you. And some flowers!” “Open the door and come in, my dear,” said the Wolf."},
    {"image": "lrrh_wolf5.jpg", "text": "The wolf put on grandmother’s clothes and got into her bed. When Little Red Riding Hood arrived she noticed something strange. “Grandmother, what big ears you have!” “All the better to hear you with!” “What big eyes you have!” “All the better to see you with!” “What big teeth you have!” “All the better to eat you with!” And the wolf jumped up and ate her too!"},

    {"image": "sw_title1.jpg",         "text": "Snow White and the Seven Dwarfs"},
    {"image": "sw_baby2.jpg",        "text": "Once upon a time there was a little princess… She was called Snow White because she was white as snow. Her lips were red as blood. And her hair was black as night. She was a beautiful girl, sweet and gentle.  She lived in a palace with her father and her stepmother: the King and the Queen. The Queen was a beautiful woman. But she was wicked and vain."},
    {"image": "sw_mirror3.jpg",        "text": "Every day, the Queen looked in her magic mirror and asked: “Mirror, mirror on the wall,  Who’s the fairest of all?” And every day, the mirror answered: “You, my Queen! You are! You are the most beautiful of all.” Time passed and little Snow White became a beautiful young lady. Everybody loved her.  One morning, when the Queen asked her magic mirror: “Mirror, mirror on the wall, Who’s the fairest of all?”"},
    {"image": "sw_manscold4.jpg",      "text": "The mirror answered: “My Queen, you are beautiful, it’s true! But Snow White is more beautiful than you.” The Queen was furious! She planned a horrible crime. She called a huntsman and commanded him: “Take Snow White deep into the forest… and kill her!” “But… Your Majesty! The beloved Princess!” he said. “Kill Snow White! And take out her heart,” the Queen commanded again."},
    {"image": "sw_forest5.jpg",  "text": "The huntsman took Snow White deep into the woods. “Forgive me, my Princess… I have orders from the Queen to kill you! But I can’t do that!” he said. “Oh, good man!” she cried, “I’ll run into the forest and I’ll never come back!” Snow White ran and ran through the dark forest. She was so scared! Then, she found a little house. The door was open. She went in."},
    {"image": "sw_dwarfs6.jpg",        "text": "A little table was set for dinner: seven little chairs, seven little bowls, seven little spoons, seven little knives, seven little forks, and seven little mugs. Snow White was hungry and thirsty. She thought: “I’ll have a little bread from each bowl. And I’ll drink a little wine from each mug.” Then, feeling so tired, she lay down on one of the seven little beds. And she fell asleep. In the morning, she woke up and saw the seven masters of the house looking at her. They were seven dwarfs. “Who are you?” they asked. “My name is Snow White,” she answered. And she told them the whole story about the wicked Queen. “Princess Snow White, you can stay,” the Dwarfs said. “Can you help us cook and clean?” “Yes! I can cook and I can wash. And I can make beds,” she said happily. “Please, remember: don’t open the door to strangers!” said the Dwarfs. “The Queen will find you.”"},
    {"image": "sw_angry7.jpg",   "text": "At the Palace, the Queen asked her magic mirror: “Mirror, mirror on the wall, Who’s the fairest of all?” The mirror answered: “My Queen, you are beautiful, it’s true! But Snow White is more beautiful than you. In the mountains, she dwells and she is still alive and well.” The Queen was absolutely furious! “I will kill Snow White myself!” she thought. She made a deadly poison. Then, she put the poison in a beautiful red apple."},
    {"image": "sw_apple8.jpg",  "text": "The Queen, dressed in old clothes, went to the little house in the mountains. She knocked on the door, calling: “APPLES! JUICY APPLES! APPLES FOR SALE!” Snow White answered: “I’m sorry, madam. I can’t open the door to strangers.” “Oh, I’m just an old woman! Open the window and take one apple,” said the Queen. Snow White opened the window, took the apple and had one bite. She fell dead to the floor! “Now, I’m the most beautiful of all. Ha, ha, ha!” the Queen said and she ran away."},
    {"image": "sw_heppy9.jpg",   "text": "Back at the Palace, the Queen ran to her magic mirror: “Mirror, mirror on the wall, Who’s the fairest of all?” The mirror answered: “You, my Queen! You are! You are the most beautiful of all.” The wicked Queen was happy now because the magic mirror never lied."},
    {"image": "sw_hending10.jpg",   "text": "The Seven Dwarfs found Snow White dead on the floor. In her honor, they made her a glass coffin and placed it on top of a mountain. One day, a young Prince stopped to see her. And he fell in love with Snow White. With a kiss, she came back to a new life with her Prince. Snow White and the Prince had a big wedding. Everyone was invited to the celebration. Even the wicked Queen! And that night, when the Queen saw Snow White, she choked with envy and hate. And she fell dead to the floor. The Prince and the Princess lived happily ever after."}
]

IMAGE_DIR = "/content"

def create_rag_index():
    print("Loading CLIP model...")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    embeddings = []
    metadata = []

    print("Processing images and texts...")
    for idx, item in enumerate(DATA):
        img_path = os.path.join(IMAGE_DIR, item["image"])
        text = item["text"]

        if not os.path.exists(img_path):
            print(f"Warning: Image not found {img_path}, skipping.")
            continue

        try:
            image = Image.open(img_path)

            # We will use image embeddings for the index,
            # so we can retrieve stories based on image similarity (or text-to-image similarity via CLIP's shared space)
            inputs = processor(text=[text], images=image, return_tensors="pt", padding=True).to(device)

            with torch.no_grad():
                # Get image features for the index
                image_features = model.get_image_features(**inputs)
                # Normalize
                image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)

            embeddings.append(image_features.cpu().numpy())
            metadata.append({
                "id": idx,
                "image_path": img_path,
                "text": text
            })

        except Exception as e:
            print(f"Error processing {img_path}: {e}")

    if not embeddings:
        print("No data processed.")
        return

    embeddings = np.vstack(embeddings).astype('float32')

    # Create FAISS index
    print("Building FAISS index...")
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension) # Inner Product (cosine similarity since normalized)
    index.add(embeddings)

    # Save index and metadata
    os.makedirs("rag_index", exist_ok=True)
    faiss.write_index(index, "rag_index/index.faiss")
    with open("rag_index/metadata.pkl", "wb") as f:
        pickle.dump(metadata, f)

    print(f"Index created with {len(metadata)} items. Saved to 'rag_index/'")

if __name__ == "__main__":
    create_rag_index()

Loading CLIP model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Processing images and texts...


/tmp/ipython-input-1028890509.py:59: UserWarning: The following named arguments are not valid for `CLIPModel.get_image_features` and were ignored: 'input_ids', 'attention_mask'
  image_features = model.get_image_features(**inputs)
Token indices sequence length is longer than the specified maximum sequence length for this model (79 > 77). Running this sequence through the model will result in indexing errors


Building FAISS index...
Index created with 15 items. Saved to 'rag_index/'


## RAG model

In [ ]:
import os
import pickle
import torch
from transformers import CLIPProcessor, CLIPModel, BlipProcessor, BlipForConditionalGeneration
from diffusers import StableDiffusionPipeline
import faiss
import numpy as np
from PIL import Image

class MultimodalRAG:
    def __init__(self, index_dir="rag_index"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Initializing Multimodal RAG on {self.device}...")

        # Load Retrieval Index
        try:
            self.index = faiss.read_index(os.path.join(index_dir, "index.faiss"))
            with open(os.path.join(index_dir, "metadata.pkl"), "rb") as f:
                self.metadata = pickle.load(f)
            print("Loaded FAISS index and metadata.")
        except Exception as e:
            print(f"Could not load index: {e}. Retrieval will fail.")
            self.index = None
            self.metadata = []

        # Load CLIP for Retrieval
        self.clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(self.device)
        self.clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

        # Load BLIP for Image-to-Text
        print("Loading BLIP model...")
        self.blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
        self.blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(self.device)

        # Load Stable Diffusion for Text-to-Image
        print("Loading Stable Diffusion...")
        # Start with standard SD, fine-tuning will update weights later
        # We use float16 if cuda is available to save memory
        dtype = torch.float16 if self.device == "cuda" else torch.float32
        self.sd_pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=dtype).to(self.device)
        # Disable safety checker for avoiding warnings on benign story content if needed
        self.sd_pipe.safety_checker = None

    def retrieve(self, query, top_k=2):
        """
        Retrieve relevant items based on query (text or image).
        """
        if self.index is None:
            return []

        # Embed query using CLIP
        if isinstance(query, str):
            inputs = self.clip_processor(text=[query], return_tensors="pt", padding=True).to(self.device)
            with torch.no_grad():
                query_features = self.clip_model.get_text_features(**inputs)
        elif isinstance(query, Image.Image):
            inputs = self.clip_processor(images=query, return_tensors="pt").to(self.device)
            with torch.no_grad():
                query_features = self.clip_model.get_image_features(**inputs)
        else:
            raise ValueError("Query must be string or PIL Image")

        # Normalize
        query_features = query_features / query_features.norm(p=2, dim=-1, keepdim=True)
        query_vector = query_features.cpu().numpy().astype('float32')

        # Search
        scores, indices = self.index.search(query_vector, top_k)

        results = []
        for idx in indices[0]:
            if idx < len(self.metadata):
                results.append(self.metadata[idx])

        return results

    def generate_story(self, image):
        """
        Image to Text: RAG + BLIP
        """
        # 1. Retrieve similar existing contexts (optional, can be used to prompt BLIP if we do custom prompting)
        retrieved = self.retrieve(image)
        context_text = ""
        if retrieved:
            # We can use the text from the most similar image as context hint
            context_text = f"Context: {retrieved[0]['text'][:100]}... "

        # 2. Generate
        # Standard BLIP generation
        inputs = self.blip_processor(image, return_tensors="pt").to(self.device)

        # We can try to condition it more effectively if we had a fine-tuned model that accepts context.
        # For standard BLIP, we just generate valid caption/story.
        # To make it 'story-like', we might trust fine-tuning or prompt engineering.
        # Here we rely on the model's capability.

        out = self.blip_model.generate(**inputs, max_new_tokens=100)
        story = self.blip_processor.decode(out[0], skip_special_tokens=True)

        return story, retrieved

    def generate_story_from_sequence(self, images):
        """
        Generate a story from a sequence of images.
        """
        full_story = []
        all_retrieved = []

        previous_context = ""

        for i, image in enumerate(images):
            # We can optionally use the previous part of the story as context for the next
            # But BLIP basic usage is image-conditional.
            # We will generate individual segments and stitch them.

            # Retrieve
            retrieved = self.retrieve(image)
            all_retrieved.extend(retrieved)

            # Generate
            inputs = self.blip_processor(image, return_tensors="pt").to(self.device)
            out = self.blip_model.generate(**inputs, max_new_tokens=100)
            segment = self.blip_processor.decode(out[0], skip_special_tokens=True)

            full_story.append(segment)

        return " ".join(full_story), all_retrieved

    def generate_image(self, text):
        """
        Text to Image: RAG + Stable Diffusion
        """
        # 1. Retrieve similar concepts
        retrieved = self.retrieve(text)

        # 2. Augment prompt?
        # If we found a specific style or object in retrieval, we might append it.
        # For now, we pass the text directly to SD.

        prompt = text
        if retrieved:
             prompt += f", in the style of {retrieved[0]['text'][:20]}" # Heuristic augmentation

        image = self.sd_pipe(prompt).images[0]
        return image, retrieved

    def generate_image_sequence(self, story_text):
        """
        Text to Image Sequence: Generates an image for each sentence in the story.
        """
        # Split text into sentences (heuristic split by period or newline)
        import re
        sentences = [s.strip() for s in re.split(r'(?<=[.!?])\s+', story_text) if s.strip()]

        generated_images = []
        all_retrieved = []

        print(f"Generating {len(sentences)} images for the story sequence...")

        for i, sentence in enumerate(sentences):
            print(f"  Generating image for segment {i+1}: '{sentence}'")
            img, retrieved = self.generate_image(sentence)
            generated_images.append(img)
            all_retrieved.extend(retrieved)

        return generated_images, all_retrieved, sentences

## Fine-Tuning

In [ ]:
import argparse
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BlipProcessor, BlipForConditionalGeneration
from diffusers import StableDiffusionPipeline, DDPMScheduler, UNet2DConditionModel, AutoencoderKL
from transformers import CLIPTextModel, CLIPTokenizer
from PIL import Image
import os
import json
# from create_dataset import DATA, IMAGE_DIR # Commented out, DATA and IMAGE_DIR are globally available from a previous cell
from tqdm import tqdm

# Assuming DATA and IMAGE_DIR are defined in a previous cell and accessible globally.
# If not, they would need to be passed as arguments or redefined here.

# --- Dataset Class ---
class StoryDataset(Dataset):
    def __init__(self, data, processor=None, tokenizer=None, mode="blip"):
        self.data = data
        self.mode = mode
        self.processor = processor # For BLIP
        self.tokenizer = tokenizer # For SD

        # For SD image transforms
        from torchvision import transforms
        self.transforms = transforms.Compose([
            transforms.Resize((512, 512)),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5]),
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img_path = os.path.join(IMAGE_DIR, item["image"])
        image = Image.open(img_path).convert("RGB")
        text = item["text"]

        if self.mode == "blip":
            # Prepare for BLIP (Image Captioning / Story Generation)
            encoding = self.processor(images=image, text=text, padding="max_length", return_tensors="pt")
            # remove batch dimension
            return {k: v.squeeze(0) for k, v in encoding.items()}

        elif self.mode == "sd":
            # Prepare for Stable Diffusion
            pixel_values = self.transforms(image)
            input_ids = self.tokenizer(
                text,
                padding="max_length",
                truncation=True,
                max_length=77,
                return_tensors="pt"
            ).input_ids[0]

            return {"pixel_values": pixel_values, "input_ids": input_ids}

# --- Training Functions ---

def train_blip(epochs=5, batch_size=2):
    print("--- Fine-tuning BLIP (Image -> Text) ---")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

    dataset = StoryDataset(DATA, processor=processor, mode="blip")
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    model.train()

    for epoch in range(epochs):
        loop = tqdm(dataloader, desc=f"Epoch {epoch+1}")
        for batch in loop:
            input_ids = batch.pop("input_ids").to(device)
            pixel_values = batch.pop("pixel_values").to(device)

            outputs = model(input_ids=input_ids, pixel_values=pixel_values, labels=input_ids)
            loss = outputs.loss

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            loop.set_postfix(loss=loss.item())

    print("Saving fine-tuned BLIP model...")
    model.save_pretrained("fine_tuned_blip")
    processor.save_pretrained("fine_tuned_blip")


def train_sd(epochs=5, batch_size=1):
    print("--- Fine-tuning Stable Diffusion (Text -> Image) ---")
    # simplified training loop - optimizing UNet only
    device = "cuda" if torch.cuda.is_available() else "cpu"

    model_id = "CompVis/stable-diffusion-v1-4"
    tokenizer = CLIPTokenizer.from_pretrained(model_id, subfolder="tokenizer")
    text_encoder = CLIPTextModel.from_pretrained(model_id, subfolder="text_encoder").to(device)
    vae = AutoencoderKL.from_pretrained(model_id, subfolder="vae").to(device)
    unet = UNet2DConditionModel.from_pretrained(model_id, subfolder="unet").to(device)
    scheduler = DDPMScheduler.from_pretrained(model_id, subfolder="scheduler")

    # Freeze VAE and Text Encoder
    vae.requires_grad_(False)
    text_encoder.requires_grad_(False)

    dataset = StoryDataset(DATA, tokenizer=tokenizer, mode="sd")
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    optimizer = torch.optim.AdamW(unet.parameters(), lr=1e-5)

    for epoch in range(epochs):
        loop = tqdm(dataloader, desc=f"Epoch {epoch+1}")
        for batch in loop:
            # 1. Convert images to latents
            pixel_values = batch["pixel_values"].to(device)
            latents = vae.encode(pixel_values).latent_dist.sample()
            latents = latents * 0.18215 # scaling factor

            # 2. Sample noise
            noise = torch.randn_like(latents)
            bsz = latents.shape[0]
            timesteps = torch.randint(0, scheduler.num_train_timesteps, (bsz,), device=device).long()

            # 3. Add noise
            noisy_latents = scheduler.add_noise(latents, noise, timesteps)

            # 4. Get text embeddings
            input_ids = batch["input_ids"].to(device)
            encoder_hidden_states = text_encoder(input_ids)[0]

            # 5. Predict noise residual
            model_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample

            # 6. Loss
            loss = torch.nn.functional.mse_loss(model_pred.float(), noise.float(), reduction="mean")

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            loop.set_postfix(loss=loss.item())

    print("Saving fine-tuned Split-UNet for SD...")
    unet.save_pretrained("fine_tuned_sd_unet")


if __name__ == "__main__":
    train_blip(epochs=3) # Reduced for demo
    # train_sd(epochs=1) # Uncomment if GPU memory allows, SD training is heavy

--- Fine-tuning BLIP (Image -> Text) ---


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]


Epoch 1:   0%|          | 0/8 [00:00<?, ?it/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.

Epoch 3: 100%|██████████| 8/8 [02:46<00:00, 20.84s/it, loss=6.69]


Saving fine-tuned BLIP model...


## Demo

In [7]:
!pip install faiss-cpu
import os
from PIL import Image
import torch
import faiss
import numpy as np
import pickle
from transformers import CLIPProcessor, CLIPModel

# --- Redefine DATA and IMAGE_DIR locally for robustness if index needs recreation ---
DATA = [
    {"image": "lrrh_title1.jpg",       "text": "Little Red Riding Hood"},
    {"image": "lrrh_mother2.jpg",      "text": "Long long ago, in a small house near the forest, there lived a little girl with her mother. Her name was Little Red Riding Hood. One day her mother said, “Grandma is sick. She’s in bed. Go to her house. In the basket, there’s a bottle of fresh milk, some bread and some butter for her.”"},
    {"image": "lrrh_forest3.jpg",   "text": "Little Red Riding Hood walked through the forest carrying the basket. Suddenly a big bad wolf appeared. “Where are you going, little girl?” asked the wolf. “To see my grandmother,” said innocent Little Red Riding Hood; “she’s sick. I have some milk, bread and butter for her.” “Pick some flowers,” said the Wolf. And he ran to grandmother’s house"},
    {"image": "lrrh_granwolf4.jpg", "text": "The Wolf got to grandmother’s house. He went tap-tap-tap on the door. “Who is this?” asked Grandma. “It’s me, Little Red Riding Hood,” said the Wolf; “I have some milk, bread and butter for you.” “Open the door and come in, my dear,” said Grandma. The Wolf opened the door and, in an instant, he ate her up! Then, he got into grandma’s bed to wait for Little Red Riding Hood. Little Red Riding Hood got to grandmother’s house. She went tap-tap-tap on the door. “Who is this?” asked the Wolf. “It’s me, Little Red Riding Hood. I have some milk, bread and butter for you. And some flowers!” “Open the door and come in, my dear,” said the Wolf."},
    {"image": "lrrh_wolf5.jpg", "text": "The wolf put on grandmother’s clothes and got into her bed. When Little Red Riding Hood arrived she noticed something strange. “Grandmother, what big ears you have!” “All the better to hear you with!” “What big eyes you have!” “All the better to see you with!” “What big teeth you have!” “All the better to eat you with!” And the wolf jumped up and ate her too!"},

    {"image": "sw_title1.jpg",         "text": "Snow White and the Seven Dwarfs"},
    {"image": "sw_baby2.jpg",        "text": "Once upon a time there was a little princess… She was called Snow White because she was white as snow. Her lips were red as blood. And her hair was black as night. She was a beautiful girl, sweet and gentle.  She lived in a palace with her father and her stepmother: the King and the Queen. The Queen was a beautiful woman. But she was wicked and vain."},
    {"image": "sw_mirror3.jpg",        "text": "Every day, the Queen looked in her magic mirror and asked: “Mirror, mirror on the wall,  Who’s the fairest of all?” And every day, the mirror answered: “You, my Queen! You are! You are the most beautiful of all.” Time passed and little Snow White became a beautiful young lady. Everybody loved her.  One morning, when the Queen asked her magic mirror: “Mirror, mirror on the wall, Who’s the fairest of all?”"},
    {"image": "sw_manscold4.jpg",      "text": "The mirror answered: “My Queen, you are beautiful, it’s true! But Snow White is more beautiful than you.” The Queen was furious! She planned a horrible crime. She called a huntsman and commanded him: “Take Snow White deep into the forest… and kill her!” “But… Your Majesty! The beloved Princess!” he said. “Kill Snow White! And take out her heart,” the Queen commanded again."},
    {"image": "sw_forest5.jpg",  "text": "The huntsman took Snow White deep into the woods. “Forgive me, my Princess… I have orders from the Queen to kill you! But I can’t do that!” he said. “Oh, good man!” she cried, “I’ll run into the forest and I’ll never come back!” Snow White ran and ran through the dark forest. She was so scared! Then, she found a little house. The door was open. She went in."},
    {"image": "sw_dwarfs6.jpg",        "text": "A little table was set for dinner: seven little chairs, seven little bowls, seven little spoons, seven little knives, seven little forks, and seven little mugs. Snow White was hungry and thirsty. She thought: “I’ll have a little bread from each bowl. And I’ll drink a little wine from each mug.” Then, feeling so tired, she lay down on one of the seven little beds. And she fell asleep. In the morning, she woke up and saw the seven masters of the house looking at her. They were seven dwarfs. “Who are you?” they asked. “My name is Snow White,” she answered. And she told them the whole story about the wicked Queen. “Princess Snow White, you can stay,” the Dwarfs said. “Can you help us cook and clean?” “Yes! I can cook and I can wash. And I can make beds,” she said happily. “Please, remember: don’t open the door to strangers!” said the Dwarfs. “The Queen will find you.”"},
    {"image": "sw_angry7.jpg",   "text": "At the Palace, the Queen asked her magic mirror: “Mirror, mirror on the wall, Who’s the fairest of all?” The mirror answered: “My Queen, you are beautiful, it’s true! But Snow White is more beautiful than you. In the mountains, she dwells and she is still alive and well.” The Queen was absolutely furious! “I will kill Snow White myself!” she thought. She made a deadly poison. Then, she put the poison in a beautiful red apple."},
    {"image": "sw_apple8.jpg",  "text": "The Queen, dressed in old clothes, went to the little house in the mountains. She knocked on the door, calling: “APPLES! JUICY APPLES! APPLES FOR SALE!” Snow White answered: “I’m sorry, madam. I can’t open the door to strangers.” “Oh, I’m just an old woman! Open the window and take one apple,” said the Queen. Snow White opened the window, took the apple and had one bite. She fell dead to the floor! “Now, I’m the most beautiful of all. Ha, ha, ha!” the Queen said and she ran away."},
    {"image": "sw_heppy9.jpg",   "text": "Back at the Palace, the Queen ran to her magic mirror: “Mirror, mirror on the wall, Who’s the fairest of all?” The mirror answered: “You, my Queen! You are! You are the most beautiful of all.” The wicked Queen was happy now because the magic mirror never lied."},
    {"image": "sw_hending10.jpg",   "text": "The Seven Dwarfs found Snow White dead on the floor. In her honor, they made her a glass coffin and placed it on top of a mountain. One day, a young Prince stopped to see her. And he fell in love with Snow White. With a kiss, she came back to a new life with her Prince. Snow White and the Prince had a big wedding. Everyone was invited to the celebration. Even the wicked Queen! And that night, when the Queen saw Snow White, she choked with envy and hate. And she fell dead to the floor. The Prince and the Princess lived happily ever after."}
]

IMAGE_DIR = "/content"

def create_rag_index():
    print("Loading CLIP model...")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    embeddings = []
    metadata = []

    print("Processing images and texts...")
    for idx, item in enumerate(DATA):
        img_path = os.path.join(IMAGE_DIR, item["image"])
        text = item["text"]

        if not os.path.exists(img_path):
            print(f"Warning: Image not found {img_path}, skipping.")
            continue

        try:
            image = Image.open(img_path)

            inputs = processor(text=[text], images=image, return_tensors="pt", padding=True).to(device)

            with torch.no_grad():
                image_features = model.get_image_features(**inputs)
                image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)

            embeddings.append(image_features.cpu().numpy())
            metadata.append({
                "id": idx,
                "image_path": img_path,
                "text": text
            })

        except Exception as e:
            print(f"Error processing {img_path}: {e}")

    if not embeddings:
        print("No data processed.")
        return

    embeddings = np.vstack(embeddings).astype('float32')

    print("Building FAISS index...")
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    index.add(embeddings)

    os.makedirs("rag_index", exist_ok=True)
    faiss.write_index(index, "rag_index/index.faiss")
    with open("rag_index/metadata.pkl", "wb") as f:
        pickle.dump(metadata, f)

    print(f"Index created with {len(metadata)} items. Saved to 'rag_index/'")

# --- End of local definitions ---


# Assuming MultimodalRAG is defined in previous cells

class MultimodalRAG:
    def __init__(self, index_dir="rag_index"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Initializing Multimodal RAG on {self.device}...")

        # Load Retrieval Index
        try:
            self.index = faiss.read_index(os.path.join(index_dir, "index.faiss"))
            with open(os.path.join(index_dir, "metadata.pkl"), "rb") as f:
                self.metadata = pickle.load(f)
            print("Loaded FAISS index and metadata.")
        except Exception as e:
            print(f"Could not load index: {e}. Retrieval will fail.")
            self.index = None
            self.metadata = []

        # Load CLIP for Retrieval
        self.clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(self.device)
        self.clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

        # Load BLIP for Image-to-Text
        print("Loading BLIP model...")
        from transformers import BlipProcessor, BlipForConditionalGeneration
        self.blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
        self.blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(self.device)

        # Load Stable Diffusion for Text-to-Image
        print("Loading Stable Diffusion...")
        from diffusers import StableDiffusionPipeline
        # We use float16 if cuda is available to save memory
        dtype = torch.float16 if self.device == "cuda" else torch.float32
        self.sd_pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=dtype).to(self.device)
        # Disable safety checker for avoiding warnings on benign story content if needed
        self.sd_pipe.safety_checker = None

    def retrieve(self, query, top_k=2):
        """
        Retrieve relevant items based on query (text or image).
        """
        if self.index is None:
            return []

        # Embed query using CLIP
        if isinstance(query, str):
            inputs = self.clip_processor(text=[query], return_tensors="pt", padding=True).to(self.device)
            with torch.no_grad():
                query_features = self.clip_model.get_text_features(**inputs)
        elif isinstance(query, Image.Image):
            inputs = self.clip_processor(images=query, return_tensors="pt").to(self.device)
            with torch.no_grad():
                query_features = self.clip_model.get_image_features(**inputs)
        else:
            raise ValueError("Query must be string or PIL Image")

        # Normalize
        query_features = query_features / query_features.norm(p=2, dim=-1, keepdim=True)
        query_vector = query_features.cpu().numpy().astype('float32')

        # Search
        scores, indices = self.index.search(query_vector, top_k)

        results = []
        for idx in indices[0]:
            if idx < len(self.metadata):
                results.append(self.metadata[idx])

        return results

    def generate_story(self, image):
        """
        Image to Text: RAG + BLIP
        """
        # 1. Retrieve similar existing contexts (optional, can be used to prompt BLIP if we do custom prompting)
        retrieved = self.retrieve(image)
        context_text = ""
        if retrieved:
            # We can use the text from the most similar image as context hint
            context_text = f"Context: {retrieved[0]['text'][:100]}... "

        # 2. Generate
        # Standard BLIP generation
        inputs = self.blip_processor(image, return_tensors="pt").to(self.device)

        # We can try to condition it more effectively if we had a fine-tuned model that accepts context.
        # For standard BLIP, we just generate valid caption/story.
        # To make it 'story-like', we might trust fine-tuning or prompt engineering.
        # Here we rely on the model's capability.

        out = self.blip_model.generate(**inputs, max_new_tokens=100)
        story = self.blip_processor.decode(out[0], skip_special_tokens=True)

        return story, retrieved

    def generate_story_from_sequence(self, images):
        """
        Generate a story from a sequence of images.
        """
        full_story = []
        all_retrieved = []

        previous_context = ""

        for i, image in enumerate(images):
            # We can optionally use the previous part of the story as context for the next
            # But BLIP basic usage is image-conditional.
            # We will generate individual segments and stitch them.

            # Retrieve
            retrieved = self.retrieve(image)
            all_retrieved.extend(retrieved)

            # Generate
            inputs = self.blip_processor(image, return_tensors="pt").to(self.device)
            out = self.blip_model.generate(**inputs, max_new_tokens=100)
            segment = self.blip_processor.decode(out[0], skip_special_tokens=True)

            full_story.append(segment)

        return " ".join(full_story), all_retrieved

    def generate_image(self, text):
        """
        Text to Image: RAG + Stable Diffusion
        """
        # 1. Retrieve similar concepts
        retrieved = self.retrieve(text)

        # 2. Augment prompt?
        # If we found a specific style or object in retrieval, we might append it.
        # For now, we pass the text directly to SD.

        prompt = text
        if retrieved:
             prompt += f", in the style of {retrieved[0]['text'][:20]}" # Heuristic augmentation

        image = self.sd_pipe(prompt).images[0]
        return image, retrieved

    def generate_image_sequence(self, story_text):
        """
        Text to Image Sequence: Generates an image for each sentence in the story.
        """
        # Split text into sentences (heuristic split by period or newline)
        import re
        sentences = [s.strip() for s in re.split(r'(?<=[.!?])\s+', story_text) if s.strip()]

        generated_images = []
        all_retrieved = []

        print(f"Generating {len(sentences)} images for the story sequence...")

        for i, sentence in enumerate(sentences):
            print(f"  Generating image for segment {i+1}: '{sentence}'")
            img, retrieved = self.generate_image(sentence)
            generated_images.append(img)
            all_retrieved.extend(retrieved)

        return generated_images, all_retrieved, sentences

def demo():
    # Ensure index exists
    if not os.path.exists("rag_index"):
        print("Index not found, creating...")
        create_rag_index()

    rag = MultimodalRAG()

    # --- Test 1: Image to Story ---
    print("\n--- TEST 1: Image to Story ---")
    test_img_name = "lrrh_forest3.jpg" # Red Riding Hood in forest
    test_img_path = os.path.join(IMAGE_DIR, test_img_name)

    if os.path.exists(test_img_path):
        image = Image.open(test_img_path).convert("RGB")
        story, retrieved = rag.generate_story(image)

        print(f"Input Image: {test_img_name}")
        print("Retrieved Context:")
        for r in retrieved:
            print(f" - Image: {os.path.basename(r['image_path'])}, Text Snippet: {r['text'][:50]}...")

        print("\nGenerated Story:")
        print(story)
    else:
        print(f"Test image {test_img_name} not found.")

    # --- Test 2: Text to Image ---
    print("\n--- TEST 2: Text to Image ---")
    query_text = "The wicked Queen looking in the mirror"

    print(f"Query: {query_text}")
    print("Generating image (this might take a moment)...")

    # Pass 'mps' or 'cuda' or 'cpu' handling inside rag_model
    generated_image, retrieved = rag.generate_image(query_text)

    print("Retrieved Context:")
    for r in retrieved:
        print(f" - Image: {os.path.basename(r['image_path'])}, Text Snippet: {r['text'][:50]}...")

    output_path = "demo_output_queen.jpg"
    generated_image.save(output_path)
    print(f"Image saved to {output_path}")

    # --- Test 3: Story from Sequence ---
    print("\n--- TEST 3: Story from Image Sequence ---")

    # Check for a 'test_inputs' directory
    test_input_dir = "/content/test_inputs"
    sequence_images = []

    if os.path.exists(test_input_dir):
        print(f"Loading images from '{test_input_dir}'...")
        # Get all valid image files
        valid_exts = {".jpg", ".jpeg", ".png", ".bmp"}
        files = [f for f in os.listdir(test_input_dir) if os.path.splitext(f)[1].lower() in valid_exts]

        # Sort them. Try to sort numerically if names are integers (1.jpg, 2.jpg), otherwise text sort
        try:
            files.sort(key=lambda x: int(os.path.splitext(x)[0]))
        except ValueError:
            files.sort()

        print(f"Found {len(files)} images in order: {files}")
        sequence_images = [os.path.join(test_input_dir, f) for f in files]

    else:
        # Fallback to existing logic if no folder
        print(f"Directory '{test_input_dir}' not found. Using default internal list.")
        sequence_images_names = ["lrrh_forest3.jpg", "lrrh_wolf5.jpg"]
        sequence_images = [os.path.join(IMAGE_DIR, f) for f in sequence_images_names]

    img_objs = []
    for p in sequence_images:
        if os.path.exists(p):
            img_objs.append(Image.open(p).convert("RGB"))

    if img_objs:
        seq_story, _ = rag.generate_story_from_sequence(img_objs)
        print("\nGenerated Sequence Story:")
        print(seq_story)
    else:
        print("No images found for sequence test.")

    # --- Test 4: Story to Image Sequence (Unseen Data) ---
    print("\n--- TEST 4: Story to Image Sequence (Unseen Data) ---")
    kitten_story = """Once there was a tiny gray kitten who lived in a big red barn.\nEvery day he watched the golden sun rise over the green fields.\nOne morning he found a little silver bell lying in the grass.\nWhen he rang it, all the farm animals came running to play.\nFrom that day on, the kitten and his friends danced together every morning under the sun."""

    print("Input Story:")
    print(kitten_story)

    generated_images, _, sentences = rag.generate_image_sequence(kitten_story)

    # Save results
    os.makedirs("generated_sequences", exist_ok=True)
    for i, img in enumerate(generated_images):
        save_path = f"generated_sequences/kitten_part_{i+1}.jpg"
        img.save(save_path)
        print(f"Saved {save_path} for segment: '{sentences[i][:30]}...'\n")

if __name__ == "__main__":
    demo()


Initializing Multimodal RAG on cpu...
Loaded FAISS index and metadata.
Loading BLIP model...
Loading Stable Diffusion...


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]


--- TEST 1: Image to Story ---
Input Image: lrrh_forest3.jpg
Retrieved Context:
 - Image: lrrh_forest3.jpg, Text Snippet: Little Red Riding Hood walked through the forest c...
 - Image: lrrh_title1.jpg, Text Snippet: Little Red Riding Hood...

Generated Story:
a little red riding a big black bear

--- TEST 2: Text to Image ---
Query: The wicked Queen looking in the mirror
Generating image (this might take a moment)...


  0%|          | 0/50 [00:00<?, ?it/s]

Retrieved Context:
 - Image: sw_mirror3.jpg, Text Snippet: Every day, the Queen looked in her magic mirror an...
 - Image: sw_angry7.jpg, Text Snippet: At the Palace, the Queen asked her magic mirror: “...
Image saved to demo_output_queen.jpg

--- TEST 3: Story from Image Sequence ---
Loading images from '/content/test_inputs'...
Found 3 images in order: ['1.jpg', '2.jpg', '3.jpg']

Generated Sequence Story:
a man handing a child a red apple a boy and girl are playing with a bowl a cartoon of a man sitting in a chair with a child

--- TEST 4: Story to Image Sequence (Unseen Data) ---
Input Story:
Once there was a tiny gray kitten who lived in a big red barn.
Every day he watched the golden sun rise over the green fields.
One morning he found a little silver bell lying in the grass.
When he rang it, all the farm animals came running to play.
From that day on, the kitten and his friends danced together every morning under the sun.
Generating 5 images for the story sequence...
  Generatin

...

  Generating image for segment 4: 'When he rang it, all the farm animals came running to play.'


  0%|          | 0/50 [00:00<?, ?it/s]

  Generating image for segment 5: 'From that day on, the kitten and his friends danced together every morning under the sun.'


  0%|          | 0/50 [00:00<?, ?it/s]

Saved generated_sequences/kitten_part_1.jpg for segment: 'Once there was a tiny gray kit...'

Saved generated_sequences/kitten_part_2.jpg for segment: 'Every day he watched the golde...'

Saved generated_sequences/kitten_part_3.jpg for segment: 'One morning he found a little ...'

Saved generated_sequences/kitten_part_4.jpg for segment: 'When he rang it, all the farm ...'

Saved generated_sequences/kitten_part_5.jpg for segment: 'From that day on, the kitten a...'

